In [9]:
# magic command to reload modules evertime a cell with import is run
%reload_ext autoreload
%autoreload 2

In [10]:
from src.data import load_raw_data
# only 11 months of 2012, bc December is not accessible
# 2021 is working for the whole year
rides = load_raw_data(year=2022)#, months=[1,2,3,4,5,6,7,8,9,10,11]) 
rides

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
2022-12 file is not available


,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
3252712,2022-11-30 23:17:09,144
3252713,2022-11-30 23:48:48,45
3252714,2022-11-30 23:04:36,163
3252715,2022-11-30 23:18:37,161


In [11]:
# check the array of unique values for the pickup location
rides['pickup_location_id'].unique()[1:6]

array([236, 166, 114,  68, 138])

In [14]:
#%%time uncomment to see how long it takes
from src.data import transform_raw_data_into_ts_data, add_missing_slots

ts_data = transform_raw_data_into_ts_data(rides)
ts_data

100%|██████████| 265/265 [00:02<00:00, 117.99it/s]


CPU times: user 2.34 s, sys: 1.77 s, total: 4.11 s
Wall time: 4.23 s


,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
2124235,2022-11-30 19:00:00,4,265
2124236,2022-11-30 20:00:00,5,265
2124237,2022-11-30 21:00:00,7,265
2124238,2022-11-30 22:00:00,5,265


In [15]:
#%%time uncomment to see how long it takes
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28*1, # 1 month
    step_size=24
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 265/265 [00:21<00:00, 12.06it/s]

features.shape=(81090, 674)
targets.shape=(81090,)
CPU times: user 14.9 s, sys: 6.56 s, total: 21.4 s
Wall time: 22 s


In [ ]:
# save to disk

tabular_data = features
tabular_data['target_rides_next_hour'] = targets

from src.paths import TRANSFORMED_DATA_DIR
tabular_data.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')